In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import geopandas as gpd

# Data preparation

In [2]:
data_2013 = pd.read_csv("./data/2012-2013 Solar home electricity data v2.csv", header = 1)

In [4]:
#count the number of NA (non-actual) in 'Row Quality' column:
print(f"number of NA (non-actual) row: {data_2013['Row Quality'].count()}")

number of NA (non-actual) row: 0


In [5]:
#store customer infos
data_cust = data_2013.groupby(by='Customer')
cust_info = pd.DataFrame()
cust_info['Customer ID'] = data_cust['Customer'].first()
cust_info['Generator Capacity'] = data_cust['Generator Capacity'].first()
cust_info['Postcode'] = data_cust['Postcode'].first()

In [32]:
#rearrange dataframe
meltdate = data_2013.melt(id_vars = ['Customer', 'Consumption Category', 'date'], value_vars = data_2013.columns[5:53], var_name = 'time', value_name = 'power')
unstk = meltdate.pivot_table('power', ['Customer', 'date', "time"], 'Consumption Category')

In [33]:
unstk.reset_index(level = [0, 1, 2], inplace = True)
unstk['Timestamp'] = pd.to_datetime(unstk['date'] + unstk['time'], format='%d/%m/%Y%H:%M')
unstk.pop('date')
unstk.pop('time')
df = unstk.groupby(by = 'Timestamp').sum()
#unstk.pop('Consumption Category')
df.reset_index(inplace=True)
df.set_index('Timestamp', inplace=True)
df.pop('Customer')
df

Consumption Category,CL,GC,GG
Timestamp,,,
2012-07-01 00:00:00,86.601,102.973,0.015
2012-07-01 00:30:00,95.035,83.167,0.024
2012-07-01 01:00:00,79.293,76.027,0.049
2012-07-01 01:30:00,73.399,75.627,0.027
2012-07-01 02:00:00,58.312,65.588,0.038
...,...,...,...
2013-06-30 21:30:00,4.682,160.788,0.031
2013-06-30 22:00:00,16.506,154.692,0.019
2013-06-30 22:30:00,24.024,145.467,0.008


In [ ]:
australia = gpd.read_file("../data/POA_2016_AUST.shp")